In [76]:
import pandas as pd
import numpy as np
import pyarrow
from tqdm import tqdm
import pyspark.sql.functions as F
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import StringType

from pyspark.sql.types import StructType, StructField, StringType, DateType, IntegerType


In [77]:
df1 = spark.read.parquet("/home/vinny/Documents/path_teste")

In [78]:
df1.show()

+----------------+-----+----+--------------------+---------+
|            path|start| end|          transcript|  speaker|
+----------------+-----+----+--------------------+---------+
|A23CDO549BFRGH54| 0.01|0.05|bom dia meu nome ...|  cliente|
|A23CDO549BFRGH54| 0.06| 0.1|bom dia senhor ca...|atendente|
|A23CDO549BFRGH54| 0.11|0.15|eu quero uma pizz...|  cliente|
|A23CDO549BFRGH54| 0.16|0.21|o endereco de ent...|atendente|
|A23CDO549BFRGH54| 0.22|0.23|                isso|  cliente|
|A23CDO549BFRGH54| 0.24| 0.3|fica dezenove e n...|atentende|
|A23CDO549BFRGH54| 0.31|0.34|    deseja algo mais|atentende|
|A23CDO549BFRGH54| 0.35|0.37|             so isso|  cliente|
|A23CDO549BFRGH54| 0.38|0.42|a entrega sera em...|atendente|
|A23CDO549BFRGH54| 0.43| 0.5|agradecemos a pre...|atendente|
|A23CDO549BFRGH54| 0.51|0.55|pra voce tambem o...|  cliente|
|A23CDO549BFRGH55| 0.01|0.05|bom dia meu nome ...|  cliente|
|A23CDO549BFRGH55| 0.06| 0.1|bom dia senhor ca...|atendente|
|A23CDO549BFRGH55| 0.11|

In [79]:
base = df1

In [80]:
base = base.withColumnRenamed("path", "filename")

In [83]:
base.count()

99

In [82]:
base = base.na.drop()

In [84]:
udf_capitalize = F.udf(lambda x: str(x).capitalize(), StringType())

In [85]:
base = base.withColumn("transcript", udf_capitalize("transcript"))

In [73]:
#base = base.withColumn("transcript", F.expr("CASE WHEN transcript IS NULL THEN ' ' ELSE transcript END "))

In [74]:
#base = base.withColumn("transcript", F.when(F.col('transcript')=='', '').otherwise(F.col('transcript')))

In [86]:
base.filter(base["filename"]=='A23CDO549BFRGH57').show()

+----------------+-----+----+--------------------+---------+
|        filename|start| end|          transcript|  speaker|
+----------------+-----+----+--------------------+---------+
|A23CDO549BFRGH57| 0.01|0.05|Bom dia meu nome ...|  cliente|
|A23CDO549BFRGH57| 0.06| 0.1|Bom dia senhor ca...|atendente|
|A23CDO549BFRGH57| 0.11|0.15|Eu quero uma pizz...|  cliente|
|A23CDO549BFRGH57| 0.16|0.21|O endereco de ent...|atendente|
|A23CDO549BFRGH57| 0.22|0.23|                Isso|  cliente|
|A23CDO549BFRGH57| 0.24| 0.3|Fica dezenove e n...|atentende|
|A23CDO549BFRGH57| 0.31|0.34|    Deseja algo mais|atentende|
|A23CDO549BFRGH57| 0.35|0.37|             So isso|  cliente|
|A23CDO549BFRGH57| 0.38|0.42|A entrega sera em...|atendente|
|A23CDO549BFRGH57| 0.43| 0.5|Agradecemos a pre...|atendente|
|A23CDO549BFRGH57| 0.51|0.55|Pra voce tambem o...|  cliente|
+----------------+-----+----+--------------------+---------+



In [100]:
newDataframe = base.groupby('filename', 'transcript').agg({"transcript" :". ".join}).show()

AttributeError: 'builtin_function_or_method' object has no attribute '_get_object_id'

In [55]:
def udf_concat_transc_separation(dataframe):
    
    
    coluna_transc = dataframe['transcript'] # coluna_transc = dataframe['transcript']
    coluna_grouped = dataframe['filename']
    

    #dataframe.dropna(subset = [coluna_transc], inplace = True)
    #dataframe[coluna_transc] = dataframe[coluna_transc].astype('str')
    #dataframe[coluna_transc] = np.where(dataframe[coluna_transc].isnull(), '', dataframe[coluna_transc])
    newDataframe = dataframe.groupby([coluna_grouped], as_index=False)[coluna_transc].agg({coluna_transc: '. '.join})
        
    
    return newDataframe #newDataframe

In [14]:
base.printSchema()

root
 |-- filename: string (nullable = true)
 |-- start: double (nullable = true)
 |-- end: double (nullable = true)
 |-- transcript: string (nullable = true)
 |-- speaker: string (nullable = true)



#### EXEMPLO DE CRIACAO DE FUNCAO - GK Tech - UDF, convers'ao de tipos ...

In [4]:
# EXEMPLO DO SCHEMA
schema = (StructType([
    StructField('nm_cliente', StringType(), True),
    StructField('data', StringType(), True),
    StructField('pcd_cliente', StringType(),True),
    StructField('pzo', StringType(),True)
]))

In [ ]:
# Criando a coluna dt_carga para obter a data atual
df = df.withColumn('dt_carga', F.date_formtat(F.current_timestamp(), 'dd-MM-yyyy'))

In [ ]:
#Convertendo os timpos primitivos da coluna data_d
df = df.withColumn('data_d', F.col('data_d').cast(DateType()))

In [ ]:
# transformando a coluna em uma lista para fazer o calculo da funcao date_add
list_pzo = df.select('pzo').rdd.flatmap(lambda x:x).collect()

In [ ]:
#registrando a funcao
def date_add_days(data_d, pzo):
    return data_d + timedelta(days=pzo)

In [ ]:
#Variavel registrar a funcao com UDF
date_add_function = F.udf(dateadd_days, DateType())

In [ ]:
# chamar a nossa UDF para fazer a soma de duas colunas do DF
df = df.withColumn('soma_data', date_add_function(df.data_d, df.pzo))

In [ ]:
# FAZENDO O JOIN ENTRE DOIS DATAFRAMES
df_final = df_2.join(df, df_2['cd_cliente'] == df['cd_cliente'], "inner").filter(F.col('soma_data') >= F.current_date())

In [ ]:
# Usando Expressoes SQL
df = df.withColumn("comprar", F.expr("CASE WHEN Ano < 2020 THEN 'sim' ELSE 'nao' END"))1 